In [ ]:
import pandas as pd
import numpy as np
import sklearn

train = pd.read_csv("/Users/rajeshkhadka/Documents/epita/data_science_in_production/dsp_rajesh_khadka/data/house-prices/train.csv")
test = pd.read_csv("/Users/rajeshkhadka/Documents/epita/data_science_in_production/dsp_rajesh_khadka/data/house-prices/test.csv")

In [ ]:
seed = 0

In [ ]:
%%capture
train.head()

In [ ]:
%%capture
test.head()

In [ ]:
train.isna().values.any()

In [ ]:
train = train.fillna(train.mode().iloc[0])
test = test.fillna(train.mode().iloc[0])

In [112]:
%%capture
train.corr()

### Feature Extraction

#### Correlation
Looking at the correlation values from above table. I have taken into the consideration of the feature which has correlation value more than or equal to 0.60

In [113]:
from sklearn.model_selection import train_test_split

y = train[['SalePrice']]
X = train[['OverallQual','GrLivArea','GarageArea','GarageCars','1stFlrSF','TotalBsmtSF']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

#### Dummy Model

In [114]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import LogisticRegression

random_forest = LogisticRegression()
random_forest.fit(X_train.values, y_train.values.reshape(1, -1)[0])

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [115]:
y_pred = random_forest.predict(X_test)
mean_squared_log_error(y_test, y_pred)

0.20251207183955144

### Baseline Model

#### Split Dataset

In [116]:
X = train.drop('SalePrice', axis=1)
y = train['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=seed)

####  Handle categorical data

In [117]:
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

#### Handle Numerical Data

In [118]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

#### Column Transformer for numerical and Categorical Data

In [119]:
preprocessor = ColumnTransformer([('cat', cat_pipe, categorical), ('num', num_pipe, numerical)],
                                 remainder='passthrough')
preprocessor.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['MSZoning', 'Street', 'Alley', 'LotShape',
                                  'LandContour', 'Utilities', 'LotConfig',
                                  'LandSlope', 'Neighborhood', 'Condition1',
                                  'Condition2', 'BldgT...
                                  'OverallQual', 'OverallCond', 'YearBuilt',
                                  'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
         

#### Pipeline

In [120]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression())
])
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['MSZoning', 'Street',
                                                   'Alley', 'LotShape',
                                                   'LandContour', 'Utilities',
                                                   'LotCon

In [121]:
y_train_pred = pipeline.predict(X_train)

y_test_pred = pipeline.predict(X_test)
mean_squared_log_error(y_test, y_test_pred)

0.053911165167788974

In [122]:
def create_sumission_file(data):
    df = pd.DataFrame(data, columns = ['SalePrice'])
    df.set_index(X_test.index, inplace=True)
    df.index.name = 'Id'
    df.to_csv("../sample_submission.csv",)
create_sumission_file(y_test_pred)

In [ ]:
def save_objects():
    